In [1]:
# USAGE
# python train_mask_detector.py --dataset dataset

# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

In [2]:
import tensorflow as tf

In [3]:
import argparse

ap = argparse.ArgumentParser()
ap.add_argument("-d", "--dataset", type=str, help="path to input dataset")
ap.add_argument("-p", "--plot", type=str, default="plot.png", help="path to output loss/accuracy plot")
ap.add_argument("-m", "--model", type=str, default="mask_detectorch.model", help="path to output face mask detector model")
args, unknown = ap.parse_known_args()

In [4]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 1e-4
EPOCHS = 20
BS = 32

In [5]:
from imutils import paths

print("[INFO] loading images...")
imagePaths = list(paths.list_images(r"E:\Hope AI\1.Tamil-20230710T103356Z-001\1.Tamil\Week11-Deep Learning Module\FMD Project\archive\images\Dataset"))
data = []
labels = []

# loop over the image paths
for imagePath in imagePaths:
    # extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2]

    # load the input image (224x224) and preprocess it
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)

    # update the data and labels lists, respectively
    data.append(image)
    labels.append(label)

# convert the data and labels to NumPy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)

# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

[INFO] loading images...


In [9]:
data.shape

(351, 224, 224, 3)

In [11]:
labels.shape

(4914,)

In [14]:
import numpy as np
from sklearn.model_selection import train_test_split

# Assuming you have your data stored in 'data' and labels in 'labels'
# Make sure data and labels are aligned properly

# Step 1: Align data and labels if necessary
# For example, assuming you have 'data' as a list of arrays
data = np.array(data)
labels = np.array(labels)

# Check the shape of labels and reshape if necessary
if labels.ndim > 1:
    labels = np.squeeze(labels, axis=(1, 2))  # Squeeze extra dimensions

# Verify the number of samples in data and labels
if len(data) != len(labels):
    raise ValueError("Number of samples in data and labels do not match.")

# Step 2: Split the data into training and testing sets
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.20, stratify=labels, random_state=42)

# Now you have trainX, trainY, testX, and testY ready for training and evaluation


ValueError: Number of samples in data and labels do not match.

In [26]:
# construct the training image generator for data augmentation
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

In [27]:
# load the MobileNetV2 network, ensuring the head FC layer sets are
# left off
baseModel = MobileNetV2(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))

In [28]:
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

In [29]:
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

In [30]:
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
    layer.trainable = False

In [31]:
# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
    metrics=["accuracy"])

[INFO] compiling model...


C:\Users\Admin\Anaconda3\envs\facemask\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
# train the head of the network
print("[INFO] training head...")
H = model.fit(
    aug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=EPOCHS)

[INFO] training head...
Epoch 1/20
96/96 [==============================] - 194s 2s/step - loss: 0.4281 - accuracy: 0.8351 - val_loss: 0.1667 - val_accuracy: 0.9818
Epoch 2/20
96/96 [==============================] - 166s 2s/step - loss: 0.1659 - accuracy: 0.9622 - val_loss: 0.0874 - val_accuracy: 0.9870
Epoch 3/20
96/96 [==============================] - 170s 2s/step - loss: 0.1074 - accuracy: 0.9727 - val_loss: 0.0647 - val_accuracy: 0.9896
Epoch 4/20
96/96 [==============================] - 162s 2s/step - loss: 0.0830 - accuracy: 0.9744 - val_loss: 0.0517 - val_accuracy: 0.9909
Epoch 5/20
46/96 [=============>................] - ETA: 1:08 - loss: 0.0779 - accuracy: 0.9771

In [ ]:
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

In [ ]:
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

In [ ]:
# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
    target_names=lb.classes_))

In [ ]:
# Set the desired argument values directly
args = argparse.Namespace(model="model.h5")

# The rest of your code...

# serialize the model to disk
print("[INFO] saving mask detector model...")
model.save(args.model, save_format="h5")

In [ ]:
# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_accuracy")  # Modify this line
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_accuracy")  # Modify this line
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")